In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Primeros Pasos con Vertex AI Gemini API & Python SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_python.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Ejecutar en Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_python.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> Ver en GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_python.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Abrir en Workbench de Vertex AI
    </a>
  </td>
</table>


## Visión General

### Gemini

Gemini es una familia de modelos generativos de IA desarrollados por Google DeepMind y proyectados para casos de uso multimodales. La API Gemini da acceso a los modelos Gemini Pro Vision y Gemini Pro


### API Vertex AI Gemini

La API Vertex AI Gemini nos da una interface unificada para interactuar com modelos Gemini. Actualmente existen dos modelos disponíbles en la API Gemini:


- **Modelo Gemini Pro** (`gemini-pro`): Proyectado para manejar tareas de lenguage natural, chat multiturno de texto y código y generación de código.
- **Modelo Gemini Pro Vision** (`gemini-pro-vision`): Soporta prompts multimodales. Puedes incluir texto, imágenes y video en los prompt y obtener respuestas en texto o código.

Puedes interactuar con la API Gemini usando los siguiente métodos:

- Usando [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) para pruebas rápidos y generación de comandos.
- Usando el SDK de Vertex AI

Este notebook se concntra en el uso de **Vertex AI SDK para Python** para utilizar la API Vertex AI Gemini.

Para obtener más inforción, consulte la documentación [IA Generativa en Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Objetivos

En este tutorual, aprenderás como usar la API Vertex AI Gemini con el SDK Vertex AI para Python para interactuar con los modelos Gemini Pro (`gemini-pro`) y Gemini Pro Vision (`gemini-pro-vision`).

Concluiras las siguientes tareas:

- Instalar el SDK de Vertex AI para Python
- Usar la API Vertex AI Gemini para interactuar con cada modelo
  - Modelo Gemini Pro (`gemini-pro`):
    - Genera textos a partir de prompts de texto.
    - Exploa varios recursos y opciones de configuración.
  -  Modelo Gemini Pro Vision (`gemini-pro-vision`):
    - Genera texto a partir de imágenes y prompts de texto.
    - Genera texto a partit de prompts de video y texto.



### Costos

Este tutorial usa los siguientes componentes de Google Cloud que pueden generar costos en su factura:

- Vertex AI

Para mayores detalles puedes revisar los [precios de Vertex AI](https://cloud.google.com/vertex-ai/pricing) y usar la [calculadora de precios](https://cloud.google.com/products/calculator/) para generar una estimativa de costos con base en el uso del proyecto.

## Primeros pasos

### Instala el SDK de Vertex AI
**Importante:** solo descomente la siguiente línea si **no estas** ejecutando este laboratório en Qwiklabs (Skills Boost).

In [ ]:
# ! pip3 install --upgrade --user google-cloud-aiplatform

### **Reinicie el kernel de su jupyter notebook**

Como la instalción está siendo realizada con la opción `--user` es necesario reiniciar el kernel para que los nuevos módulos se tornen accesibles.

**Importante:** solo descomente las siguientes lineas si **não estiver** ejecutando este laboratório en Qwiklabs (Skills Boost).

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ El kernel del notebook está siendo reiniciado. Por favor espere que este proceso sea finalizado para poder continuar con los próximos pasos. ⚠️</b>
</div>

### **Solamente para uso en Colab - Autentique su ambiente de notebook**

En el caso que estes ejecutando este notebook en un Google Colab, descomenta la siguiente célula para realizar la autenticación de su sesión de notebook con Google Cloud. Este paso es importante **para la utilización del Colab**, asi podemos garantizar que las llamdas a las APIs de Google Cloud funcionen sin problemas.

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if "google.colab" in sys.modules:
#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

### **Solamente para usos en Colab - define el proyecto en Google Cloud que va a ser utilizado**

En el caso que estes ejeccutando este notebook en un Google Colab, descomente la siguiente célula para definir que proyecto en Google Cloud sera utilizado por el Colab en la ejecución de este notebook. De lo contrario, continue con los próximos pasos.

In [ ]:
# if "google.colab" in sys.modules:
#     # Define project information
#     PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
#     LOCATION = "us-central1"  # @param {type:"string"}

#     # Initialize Vertex AI
#     import vertexai

#     vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importe las bibliotecas necesarias

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.simplefilter("ignore", UserWarning)

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

## Importe el modelo `Gemini 1.0 Pro`

Gemini Pro (`gemini-1.0-pro`) ayuda en la realización de tareas utilizando lenguage natural, chats multiturno de texto y código e para la generación de código.

In [ ]:
model = GenerativeModel("gemini-1.0-pro")

### Genere textos a partir de prompts de texto

Envia un prompt de texto para el modelo. El modelo Gemini Pro (`gemini-pro`) ofrece un mecanismo de respuesta en streaming. De esta manera, no necesitas esperar por la respuesta completa, puedes comenzar a procesar fragmentos tan pronto esten accesibles.

In [ ]:
responses = model.generate_content("¿Por qué el cielo es azul?", stream=True)

for response in responses:
    print(response.text, end="")

#### Prueba tus propios prompts

- ¿Cuáles son los mayores desafios que el sector de salud enfrenta?
- ¿Cuáles son los últimos avances en la industria del automóvil?
- ¿Cuáles son las mayores oportunidades en el sector minorista?
- (¡Prueba con tus propias instrucciones!)

In [ ]:
prompt = """ Crea una lista enumerada de 10 items. Cada item de la lista de ser una tendencia en la industria de la tecnología.

Cada tendencia debe tener menos de 5 palabras""" # Prueba con tu propio prompt

responses = model.generate_content(prompt, stream=True)

for response in responses:
    print(response.text, end="")

#### Parametros del modelo

Cada prompt enviado al modelo incluye valores y parametros que controlan como el modelo genera respuestas. El modelo puede generar resultados diferentes para valores de parametros diferentes. Puedes probar diferentes parametros del modelo para ver como los resultados cambian.


In [ ]:
generation_config = GenerationConfig(
    temperature=0.9, # cambie esta variable para probar diferentes temperaturas (entre 0 e 1.0)
    top_p=1.0, # cambie esta variable para probar top_p diferentes (entre 0.1 e 1.0)
    top_k=32, # cambie esta variable para probar top_k diferentes (entre 1 e 40)
    candidate_count=1,
    max_output_tokens=8192, # cambie esta variable para probar diferentes tamaños de respuesta (entre 1 y 8192 para Gemini-Pro)
)

responses = model.generate_content(
    "¿Por qué el cielo es azul?",
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

### Probemos los prompts de chat

El modelo Gemini Pro ofrece soporte a chats com varios turnos y es ideal para tareas de texto que exigen interacciones de ida y vuelta. Los siguientes ejemplos muestran como el modelo responde durante una conversación multiturnos.

In [ ]:
chat = model.start_chat()

prompt = """Mi nombre es Alvaro. Tú eres mi asistente personal. Mis películas favoritas son El señor de los Anillos y El Hobbit.

Sugiereme otra película que me podría gustar.
"""

responses = chat.send_message(prompt, stream=True)

for response in responses:
    print(response.text, end="")

Este prompt de acompañamiento muestra como el modelo responde con base en el prompt anterior:

In [ ]:
prompt = "¿Qué tienen estas películas en común con las películas que me gustan?"

responses = chat.send_message(prompt, stream=True)

for response in responses:
    print(response.text, end="")

También puedes tener acceso al histórico del chat:

In [ ]:
print(chat.history)

## Importe el modelo Gemini 1.0 Pro Vision

Gemini Pro Vision (`gemini-pro-vision`) es un modelo multimodal que soporta prompts multimodales. Puedes incluir texto, imagen(es) y video en los prompt y obtener respuestas en texto o código.

In [ ]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

### Define algunas funciones auxiliares

Define funciones auxiliares para cargar y mostrar imágenes.

In [ ]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)

def get_url_from_gcs(gcs_uri: str) -> str:
    # converts gcs uri to url for image display.
    url = "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(" ", "%20")
    return url

def print_multimodal_prompt(contents: list):
    for content in contents:
        if isinstance(content, Image):
            display_images([content])
        elif isinstance(content, Part):
            url = get_url_from_gcs(content.file_data.file_uri)
            IPython.display.display(load_image_from_url(url))
        else:
            print(content)

### Genera texto a partir de imágenes locales

Usa el método `Image.load_from_file` para cargar un archivo local como la imagen para la cual generamos texto.

In [ ]:
# Download an image from Google Cloud Storage
! gsutil cp "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg" ./image.jpg

# Load from local file
image = Image.load_from_file("image.jpg")

# Prepare contents
prompt = "Describe esta imagen"
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Respuesta--------")
for response in responses:
    print(response.text, end="")

### Generar texto a partir de texto e imágenes

#### Imágenes con URIs de Cloud Storage

Si tus imagenes están almacenadas en [Cloud Storage](https://cloud.google.com/storage/docs), puedes especificar la URI de Cloud Storage de la imágen para incluirlo en el prompt. También debes especificar el campo `mime_type`. Los tipos MIME soportados para imagenes incluyen `image/png` e `image/jpeg`.

Observe que la URI (no debe ser confundida con URL) de un objeto de Cloud Storage debe siempre comenzar con `gs://`.

In [ ]:
# Load image from Cloud Storage URI
gcs_uri = "gs://cloud-samples-data/generative-ai/image/boats.jpeg"

# Prepare contents
image = Part.from_uri(gcs_uri, mime_type="image/jpeg")
prompt = "Describe esta escena"
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, generation_config=GenerationConfig(temperature=0.0), stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Respuesta--------")
for response in responses:
    print(response.text, end="")

#### Imágenes con links directos

También puedes usar links directos para imágenes, conforme se muestra abajo. La función auxiliar `load_image_from_url()` (que fue declarada anteriormente) convierte la imagen en bytes e la retorna como un objeto Image que puede ser enviado a  `Gemini Pro Vision` junto con el prompt de texto.

In [ ]:
# Load image from Cloud Storage URI
image_url = (
    "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/boats.jpeg"
)
image = load_image_from_url(image_url) # convert to bytes

# Prepare contents
prompt = "Describe esta escena"
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, generation_config=GenerationConfig(temperature=0.0), stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Respuesta--------")
for response in responses:
    print(response.text, end="")

#### Combinando várias imágenes y texto en un prompt *few-shot*

Puedes enviar más de una imagen por vez y colocarlos en cualquier lugar al lado de um prompt de texto.

En el ejemplo, la solicitación con algunas fotos es ejecutada para que o `Gemini Pro Vision` retorne la ciudad y los puntos de referencia en un formato JSON específico.

In [ ]:
# Load images from Cloud Storage URI
image1_url = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark1.jpg"
image2_url = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark2.jpg"
image3_url = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark3.jpg"
image1 = load_image_from_url(image1_url)
image2 = load_image_from_url(image2_url)
image3 = load_image_from_url(image3_url)

# Prepare prompts
prompt1 = """{"ciudad": "Londres", "punto_turistico:", "Big Ben"}"""
prompt2 = """{"ciudad": "Paris", "punto_turistico:", "Torre Eiffel"}"""

# Prepare contents
contents = [image1, prompt1, image2, prompt2, image3]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Respuesta--------")
for response in responses:
    print(response.text, end="")

### Genere texto a partir de un archivo de video

Especifique la URI de Cloud Storage de un video que sera incluído en el prompt.
También debes especificar el campo `mime_type`. El tipo MIME soportado para videos incluye `video/mp4`.

In [ ]:
file_path = "github-repo/img/gemini/multimodality_usecases_overview/pixel8.mp4"
video_uri = f"gs://{file_path}"
video_url = f"https://storage.googleapis.com/{file_path}"

IPython.display.Video(video_url, width=450)

In [ ]:
prompt = """
Responde las siguientes preguntas usando apenas el video:
¿Cuál es la profesión de la persona principal del video?
¿Cuales son las principales funcionalidades del telefono que son presentadas en este video?
¿En qué ciudad fue grabado?

Retorna la respuesta en formato JSON.
"""

video = Part.from_uri(video_uri, mime_type="video/mp4")
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

for response in responses:
    print(response.text, end="")